In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow.keras import Input, Model
from keras.layers import Dense, Dropout, Flatten, add, BatchNormalization
import keras
from keras.applications.xception import Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomCrop, Rescaling, RandomTranslation
from keras import Sequential
from tqdm import tqdm
from tensorflow.data import Dataset
from kaggle_datasets import KaggleDatasets
import tensorflow_addons as tfa
import random
from functools import partial
from sklearn.model_selection import train_test_split
import re
seed = 456
batch_size = 16
random.seed(seed)
image_size = 512

Counting the Number of samples in the dataset

In [ ]:
root_dir = '../input/cassava-leaf-disease-classification'

train_df = pd.read_csv(os.path.join(root_dir, 'train.csv'))
print("there are " + str(train_df.shape[0]) + " train samples" )
train_df.head()

Short block to print samples of the train set, each element in a row belongs to the same category

In [ ]:
# now looking at examples of each category

#{"0": "Cassava Bacterial Blight (CBB)", 
#"1": "Cassava Brown Streak Disease (CBSD)",
#"2": "Cassava Green Mottle (CGM)", 
#"3": "Cassava Mosaic Disease (CMD)",
#"4": "Healthy"}



train_img_dir = os.path.join(root_dir, 'train_images')  

figure = plt.figure(figsize = (20,20))

cont = 0
    
for i in range(5):
    
    speci = train_df[train_df['label'] == i]
    
    for j in range(5):
        
        img = Image.open(os.path.join(train_img_dir, speci.iloc[j,0]))
        
        plt.subplot(5,5, cont+1)
        
        plt.imshow(img)
        
        cont = cont + 1

help functions for the tensorflow documentation

In [ ]:
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

Functions to read, augment and return the datasets

In [ ]:
autotune = tf.data.experimental.AUTOTUNE 

records_dir = "../input/cassava-leaf-disease-classification/train_tfrecords"

train_tfnames, valid_tfnames = train_test_split(
    tf.io.gfile.glob(records_dir + "/ld_train*.tfrec"),
    test_size=0.2, random_state=5
)

test_records_dir =  "../input/cassava-leaf-disease-classification/test_tfrecords"
tfnames_test = tf.io.gfile.glob(test_records_dir + "/ld_test*.tfrec")



def read_tf(example, labeled):
    
    """
    this function reads a record and returns its feautres
    
    """
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = tf.io.decode_jpeg(example['image'])
    image = tf.cast(image, tf.float32) / 255.0
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

    
    
def load_ds(filenames, labeled = True):
    
    tfrecords = tf.data.TFRecordDataset(filenames)
    dataset = tfrecords.map(partial(read_tf,labeled = labeled), num_parallel_calls = autotune)
    return dataset

def augment(image, label):
    angle = random.random() * 0.79 #random angle between 0 and 0.79 radians roughly 45 degrees 
    image = tf.image.random_flip_up_down(image, seed = seed) 
    image = tf.image.random_flip_left_right(image, seed = seed)
   # image = tfa.image.gaussian_filter2d(image, 2) I removed the computing burden would increase too much
    image = tfa.image.rotate(image, angle) #rotating image with a random angle
    image = tf.image.random_crop(image, [image_size,image_size,3], seed = seed) # randomly cropping a 300x300 section of the image
    
    return image, label
    
    
def augment_val(image, label):
    image = tf.image.random_crop(image, [image_size,image_size,3], seed = seed) # randomly cropping a 300x300 section of the image
    

    return image, label

def get_train_ds():
    datset = load_ds(train_tfnames)
    datset = datset.map(augment, num_parallel_calls = autotune)
    datset = datset.repeat()
    datset = datset.shuffle(seed)
    datset = datset.batch(batch_size)
    datset = datset.prefetch(autotune)
    
    return datset

def get_test_df():
    
    datset = load_ds(test_tfnames, labeled = False)
    datset = datset.map(augment, num_parallel_calls = autotune)
    datset = datset.batch(batch_size)
    datset = datset.prefetch(autotune)
    
    return datset
    

def get_val_ds():
    dataset = load_ds(valid_tfnames, labeled=True) 
    dataset = dataset.map(augment_val, num_parallel_calls = autotune)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(autotune)
    return dataset


Creating the Model, Several pretrained models were used and their output was normalized to a 512 layer to sum them in the main 

In [ ]:
#pretrained models

xception_weights_dir = '../input/xception-form-keras/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_xcep = Xception(weights = xception_weights_dir, include_top = False, input_shape = (image_size, image_size, 3))
base_xcep.trainable  = False 

#vgg16_weights_dir = "../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
#base_vgg16 = VGG16(weights = vgg16_weights_dir, include_top = False, input_shape = (image_size,image_size,3))
#base_vgg16.trainable = False

#resnet50_weights_dir =  "../input/resnet50-keras/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
#base_resnet50 = ResNet50(weights = resnet50_weights_dir, include_top = False, input_shape = (image_size,image_size,3))
#base_resnet50.trainable = False

#inceptionv3_dir = "../input/aaaaa/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
#base_incepv3 = InceptionV3(weights = inceptionv3_dir, include_top = False, input_shape = (image_size, image_size, 3))
#base_incepv3.trainable = False

efficientnet_dir = "../input/efficientnet-b0-for-keras-no-top/efficientnetb0_notop.h5"
base_efficientnet = tf.keras.applications.EfficientNetB0(weights = efficientnet_dir, include_top = False, input_shape = (image_size, image_size, 3))
base_efficientnet.trainable = False

    # function to create a base model with standard output of 1024

def create_base_model(base_model):

        inputs = Input(shape = (image_size,image_size,3))
        
        

        base_out = base_model(inputs)

        base_out = Flatten()(base_out)

        base_out = Dropout(0.3)(base_out)

        base1_out = Dense(1024, activation = 'relu')(base_out)

        model = Model(inputs = inputs, outputs = base1_out)

        return model

xcep_base = create_base_model(base_xcep)
#vgg16_base = create_base_model(base_vgg16)
#resnet50_base = create_base_model(base_resnet50)
#incep_base = create_base_model(base_incepv3)
eff_base = create_base_model(base_efficientnet)

def create_model(base1, base2):

        inputs = Input(shape = (image_size,image_size,3)) 

    
    
        base1_out = base1(inputs)
        base2_out = base2(inputs)
        
        base_out = add([base1_out, base2_out])


        outputs = Dense(5, activation = 'softmax')(base_out)

        model = Model(inputs = inputs, outputs = outputs)

        model.summary()

        model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy')

        return model



model = create_model(eff_base,xcep_base)

EfficientNet use

In [ ]:
epochs = 30
steps_per_epoch = count_data_items(train_tfnames) // batch_size
steps_per_epoch_val = count_data_items(valid_tfnames) // batch_size

call_list = [tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=False
)]

val_ds = get_val_ds()
train_ds = get_train_ds()

history = model.fit(train_ds, epochs = epochs, steps_per_epoch = steps_per_epoch, validation_data = val_ds,
                   callbacks = call_list, validation_steps = steps_per_epoch_val)


The testing stage is not done with the TFRecords because it was giving errors. Instead the images were used directly from the files. TTA applied to improve the accuracy of the output.

In [ ]:
image_preprocessor = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomCrop(image_size,image_size),
    RandomRotation(0.25),
    Rescaling(1./255)])



test_dir = "../input/cassava-leaf-disease-classification/test_images"

test_names = pd.Series(os.listdir(test_dir))


for j in range(3):

    for i in tqdm(range(len(test_names))):

        image = cv2.imread(os.path.join(test_dir, test_names[i]))
        image = np.expand_dims(image, axis = 0)
        image = image_preprocessor(image)
        if i ==0:

            pred = model.predict(image)
        else:
            pred = np.concatenate([pred, model.predict(image)])
            
    if j ==0:
        final = pred
    else:
        final = final +pred
     
pred = pd.Series(np.argmax(final, axis = 1))


test_df = pd.concat([test_names, pred], axis = 1)
test_df = test_df.rename(columns = {0: 'image_id', 1: 'label'})

test_df.to_csv('submission.csv', index = False)

